In [1]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-14.2")
user

UserTO(id=8811c104-0826-4498-8d75-423e093aa824, username=Johann-14.2)

In [2]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

In [3]:
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.fund.api.model.FundName

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, FundName("Expenses")) }
expenseFund

FundTO(id=f2bfba9a-b5fa-4d25-8f8a-42592930a8e5, name=Expenses)

In [4]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json
import com.charleskorn.kaml.Yaml

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_YAML_FILE).readText()
    val dataConfiguration = Yaml.default.decodeFromString(ReportDataConfigurationTO.serializer(), dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, distributions=[BudgetDistributionTO(default=true, from=null, groups=[GroupBudget

In [5]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else if (task is ReportViewTaskTO.Failed) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=d6daf497-1a7e-483c-9399-634a4c70c1a2, name=Expense report, fundId=f2bfba9a-b5fa-4d25-8f8a-42592930a8e5, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]))))

In [13]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-12-29")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=d6daf497-1a7e-483c-9399-634a4c70c1a2, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-12-29), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), net=-4082.94, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04), ReportDataGroupItemTO(group=home, net=-500.0), ReportDataGroupItemTO(group=shopping_services, net=0.0), ReportDataGroupItemTO(group=transport, net=-204.24), ReportDataGroupItemTO(group=fun, net=-1031.16), ReportDataGroupItemTO(group=gifts, net=0.0), ReportDataGroupItemTO(group=development, net=-42.5)]), ReportDataItemTO(timeBucket=DateInterval(from=2019-02-01, to=2019-02-28), net=-4301.32, value=ValueReportTO(start=2481.52, end=5185.08, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2509.31), ReportDataGroupItemTO(group=home, net=-609.16), ReportDataGroupItemTO(group=shopping_services,

In [9]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val net: Double)

val monthlyReportDF = report.data
    .mapNotNull { reportDataItem ->
        reportDataItem.net?.let { amount ->
        MonthlyExpense(reportDataItem.timeBucket.from, amount.negate().toDouble())
            }
    }
    .toDataFrame()

plot(monthlyReportDF) {
    x("month") {

    }
    bars {
        y("net")

        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1800 to 1000
    }
    line {
        y.constant(0)
    }
    line {
        y.constant(10000)
    }
    line {
        y.constant(20000)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="sVHl0i"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12],
"net":[4082.94,4301.32,8561.68,5486.66,8106.58,4529.1,10769.4,25034.218092,4454.4584992,6433.93,9787.68,5529.22,4552.19,5442.03]
},
"ggsize":{
"width":1800.0,
"height":1000.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"net"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":10000.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":20000.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
},{
"type":"float",
"column":"net"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("sVHl0i");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1800.0,
 height: 1000.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M124.45042393601034 570.622427223657 L124.45042393601034 570.622427223657 L239.06608472080552 570.622427223657 L342.58990736513806 570.622427223657 L457.20556814993324 570.622427223657 L568.1239495545742 570.622427223657 L682.739610339384 570.622427223657 L793.6579917440249 570.622427223657 L908.2736525288201 570.622427223657 L1022.8893133136153 570.622427223657 L1133.8076947182562 570.622427223657 L1248.4233555030514 570.622427223657 L1359.341736907707 570.622427223657 L1473.9573976925021 570.622427223657 L1588.5730584772973 570.622427223657 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M124.45042393601034 220.24485444731408 L124.45042393601034 220.24485444731408 L239.06608472080552 220.24485444731408 L342.58990736513806 220.24485444731408 L457.20556814993324 220.24485444731408 L568.1239495545742 220.24485444731408 L682.739610339384 220.24485444731408 L793.6579917440249 220.24485444731408 L908.2736525288201 220.24485444731408 L1022.8893133136153 220.24485444731408 L1133.8076947182562 220.24485444731408 L1248.4233555030514 220.24485444731408 L1359.341736907707 220.24485444731408 L1473.9573976925021 220.24485444731408 L1588.5730584772973 220.24485444731408 " fill="n

In [10]:
val categories = mapOf(
    "basic" to Color.GREEN,
    "home" to Color.PEACH,
    "shopping_services" to Color.LIGHT_GREEN,
    "transport" to Color.BLUE,
    "fun" to Color.ORANGE,
    "gifts" to Color.RED,
    "development" to Color.PURPLE,
)

fun getGroupedExpensesDataFrame(): DataFrame<*> {
    val categoriesNetExtractor: Map<String, (List<ReportDataGroupItemTO>) -> Double> =
        categories
            .keys
            .map { group ->
                group to { groups: List<ReportDataGroupItemTO> ->
                    groups.first { it.group == group }.net?.negate()?.toDouble() ?: 0.0
                }
            }
            .toMap()

    return report.data
        .map { monthlyData ->
            monthlyData.timeBucket.from to categoriesNetExtractor.mapValues { (_, extractor) -> extractor(monthlyData.groups!!) }
        }
        .let {
            dataFrameOf(
                "month" to it.flatMap { (month, values) -> values.map { month } },
                "value" to it.flatMap { (_, values) -> values.map { it.value } },
                "category" to it.flatMap { (_, values) -> values.keys }
            )
        }
}
getGroupedExpensesDataFrame().groupBy("category").plot {
    x("month")
    line {
        y("value")
        color("category") {
            scale = categorical(
                *categories.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expenses by category"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ZQTK9L"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Expenses by category"
},
"mapping":{
},
"data":{
"&merged_groups":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development","development","development"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12],
"category":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development","development","development"],
"value":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,2158.

In [12]:
data class MonthlyExpense(val month: LocalDate, val start: Double, val end: Double)

val df = report.data
    .mapNotNull { item ->
        item.value?.let { valueReport ->
            MonthlyExpense(
                item.timeBucket.from,
                valueReport.start.toDouble(),
                valueReport.end.toDouble()
            )
        }
    }
    .let {
        dataFrameOf(
            "month" to it.map { it.month }.let { it + it },
            "value" to it.map { it.start } + it.map { it.end },
            "type" to List(it.size) { "startValue" } + List(it.size) { "endValue" }
        )
    }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bNFJks"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Expense fund value"
},
"mapping":{
},
"data":{
"&merged_groups":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12],
"type":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"value":[0.0,2481.52,5185.08,11301.26,11528.2727350348,10643.764995302,11871.901585538,8638.3980216284,-6550.7530315744,-873.7461008179,4052.2818580708,6037.1321957254,12396.0035787372,18628.0469211958,2481.52,5185.08,11301.26,11526.0160226864,10643.310981436,11868.2374648476,8637.9027559568,-6550.8393119068,-872.2869408013,4051.7634872547,6037.1198616282,12394.3761693426,18628.6707953371,24174.7828379781]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#fc8452","#5470c6"],
"limits":["startValue","endValue"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"float",
"column":"value"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"11"
};
 var containerDiv = document.getElementById("bNFJks");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1200.0,
 height: 600.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M45.13672835782927 396.33680232577996 L45.13672835782927 396.33680232577996 L115.80534346352943 358.0840989840085 L179.63506033319572 316.4086425513867 L250.30367543889588 222.12754905833575 L318.692657799249 218.62814111155703 L389.36127290494915 232.2628537761737 L457.75025526530953 213.33109233581763 L528.4188703710097 263.17564429783874 L599.0874854767098 497.31685180433567 L667.476467837063 409.80562401193066 L738.1450829427631 333.8707590240839 L806.5340653031162 303.2742330464104 L877.2026804088164 205.252044789